In [1]:
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
%env CUDA_VISIBLE_DEVICES=0
import os
import sys

if os.getcwd().split("/")[-1] == "notebooks":
    %cd ..
sys.path.append(".")

import jax
import jax.numpy as jnp

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: CUDA_VISIBLE_DEVICES=0
/mnt/aitrics_ext/ext01/john/jax/Neural-Process-Family


In [2]:
from npf.jax.modules.setconv import Discretization1d
from npf.jax.data import GPSampler, Matern52Kernel, RBFKernel, PeriodicKernel

sampler = GPSampler(RBFKernel())
batch = sampler.sample(jax.random.PRNGKey(777))


In [23]:
import importlib
import math
import npf.jax.models.convnp as convnp
from npf.jax.modules import SetConv1dDecoder, SetConv1dEncoder
from npf.jax.modules.net import CNN
importlib.reload(convnp)


ConvNPBase = convnp.ConvNPBase
points_per_unit = 64
init_log_scale = math.log(2./points_per_unit)
model = ConvNPBase(z_dim=64,
                   discretizer=Discretization1d(-2, 2, points_per_unit, 1),
                   encoder = SetConv1dEncoder(init_log_scale=init_log_scale),
                   cnn = CNN(dimension=1, hidden_features=(64,64,64), out_features=64),
                   cnn_post_z= CNN(dimension=1, hidden_features=(64,64,64), out_features=64),
                   decoder = SetConv1dEncoder(init_log_scale=init_log_scale),
                   )
init_rngs = {'params':jax.random.PRNGKey(25),
             'sample':jax.random.PRNGKey(64)}
params = model.init(init_rngs, batch.x_ctx, batch.y_ctx, batch.x_tar,  batch.mask_ctx, batch.mask_tar)

In [24]:
mu, sigma = model.apply(params, 
                        batch.x_ctx, batch.y_ctx, batch.x_tar,  batch.mask_ctx, batch.mask_tar,
                        num_latents=10,
                        rngs={'sample':jax.random.PRNGKey(45)})
mu.shape

(16, 10, 50, 1)

In [25]:
model.apply(params,
            batch.x_ctx, batch.y_ctx, batch.x_tar, batch.y_tar, batch.mask_ctx, batch.mask_tar,
            num_latents=10,
            rngs={'sample':jax.random.PRNGKey(24)},
            method=model.log_likelihood
            )

DeviceArray(-2.8612857, dtype=float32)